**Stock Price Prediction using ARIMA model by Aana Kakroo**

# Data Cleaning

Import the required Python libraries

In [63]:
#importing python libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

Import datasets into Jupyter Notebook

In [64]:
#importing python module glob which is used to return all file paths that match a specific pattern
import glob

#path of the data file in the github repository clone on my local system
path = r'C:\Users\aanak\OneDrive\Desktop\Materials for college\DAC\NIFTY-Stock-Market-Prediction\data'

#using glob to get all csv files from the folder given in the path
all_files = glob.glob(path + "/*csv")
all_files

#concatenating all csv files and storing the data as a single pandas dataframe called "df"
df = pd.concat((pd.read_csv(f) for f in all_files))
df


,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Company Name,Industry,ISIN Code
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.00,962.90,984.72,27294366.0,2.687719e+15,NaN,9859619.0,0.3612,NaN,NaN,NaN
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.00,893.90,941.38,4581338.0,4.312765e+14,NaN,1453278.0,0.3172,NaN,NaN,NaN
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.00,884.20,888.09,5124121.0,4.550658e+14,NaN,1069678.0,0.2088,NaN,NaN,NaN
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.00,921.55,929.17,4609762.0,4.283257e+14,NaN,1260913.0,0.2735,NaN,NaN,NaN
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.00,969.30,965.65,2977470.0,2.875200e+14,NaN,816123.0,0.2741,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301,2021-04-26,ZEEL,EQ,188.00,190.60,191.10,185.10,186.70,186.40,187.35,8542755.0,1.600451e+14,52374.0,2340188.0,0.2739,NaN,NaN,NaN
5302,2021-04-27,ZEEL,EQ,186.40,188.00,192.95,186.80,188.80,188.15,189.41,14247767.0,2.698636e+14,73673.0,5425957.0,0.3808,NaN,NaN,NaN
5303,2021-04-28,ZEEL,EQ,188.15,188.80,190.60,187.10,188.95,189.10,188.85,8429439.0,1.591917e+14,44056.0,2413974.0,0.2864,NaN,NaN,NaN
5304,2021-04-29,ZEEL,EQ,189.10,190.80,191.65,186.00,186.60,186.55,187.44,9483009.0,1.777471e+14,60932.0,2744472.0,0.2894,NaN,NaN,NaN


Checking number of rows and columns in the dataframe

In [65]:
df.shape

(235242, 18)

Checking the datatypes of values in each column of dataframe

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235242 entries, 0 to 5305
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                235192 non-null  object 
 1   Symbol              235242 non-null  object 
 2   Series              235242 non-null  object 
 3   Prev Close          235192 non-null  float64
 4   Open                235192 non-null  float64
 5   High                235192 non-null  float64
 6   Low                 235192 non-null  float64
 7   Last                235192 non-null  float64
 8   Close               235192 non-null  float64
 9   VWAP                235192 non-null  float64
 10  Volume              235192 non-null  float64
 11  Turnover            235192 non-null  float64
 12  Trades              120344 non-null  float64
 13  Deliverable Volume  219115 non-null  float64
 14  %Deliverble         219115 non-null  float64
 15  Company Name        50 non-null     

The "Date" column is an object datatype and needs to be changed to datetime datatype

In [67]:
# converting the string to datetime format
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235242 entries, 0 to 5305
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Date                235192 non-null  datetime64[ns]
 1   Symbol              235242 non-null  object        
 2   Series              235242 non-null  object        
 3   Prev Close          235192 non-null  float64       
 4   Open                235192 non-null  float64       
 5   High                235192 non-null  float64       
 6   Low                 235192 non-null  float64       
 7   Last                235192 non-null  float64       
 8   Close               235192 non-null  float64       
 9   VWAP                235192 non-null  float64       
 10  Volume              235192 non-null  float64       
 11  Turnover            235192 non-null  float64       
 12  Trades              120344 non-null  float64       
 13  Deliverable Volume  219115 non-

Setting index of Time Series data as datetime or period makes time series forecasting easier.

In [68]:
#setting index of df as 'Date'
df.set_index('Date')

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Company Name,Industry,ISIN Code
Date,,,,,,,,,,,,,,,,,
2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.00,962.90,984.72,27294366.0,2.687719e+15,NaN,9859619.0,0.3612,NaN,NaN,NaN
2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.00,893.90,941.38,4581338.0,4.312765e+14,NaN,1453278.0,0.3172,NaN,NaN,NaN
2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.00,884.20,888.09,5124121.0,4.550658e+14,NaN,1069678.0,0.2088,NaN,NaN,NaN
2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.00,921.55,929.17,4609762.0,4.283257e+14,NaN,1260913.0,0.2735,NaN,NaN,NaN
2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.00,969.30,965.65,2977470.0,2.875200e+14,NaN,816123.0,0.2741,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-26,ZEEL,EQ,188.00,190.60,191.10,185.10,186.70,186.40,187.35,8542755.0,1.600451e+14,52374.0,2340188.0,0.2739,NaN,NaN,NaN
2021-04-27,ZEEL,EQ,186.40,188.00,192.95,186.80,188.80,188.15,189.41,14247767.0,2.698636e+14,73673.0,5425957.0,0.3808,NaN,NaN,NaN
2021-04-28,ZEEL,EQ,188.15,188.80,190.60,187.10,188.95,189.10,188.85,8429439.0,1.591917e+14,44056.0,2413974.0,0.2864,NaN,NaN,NaN


Getting statistical information about the data in the dataframe

In [69]:
df.describe()

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
count,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.00000,2.351920e+05,2.351920e+05,1.203440e+05,2.191150e+05,219115.000000
mean,1266.196349,1267.759708,1286.581440,1247.488465,1266.388302,1266.554351,1267.13230,3.045903e+06,1.610138e+14,6.196427e+04,1.315098e+06,0.502997
std,2581.370320,2585.259609,2619.649216,2546.621396,2581.392543,2582.140942,2582.69998,7.333981e+06,3.298085e+14,6.866457e+04,2.831670e+06,0.190019
min,0.000000,8.500000,9.750000,8.500000,9.100000,9.150000,9.21000,3.000000e+00,1.047000e+07,1.100000e+01,5.000000e+00,0.023600
25%,274.300000,275.000000,279.500000,269.600000,274.400000,274.350000,274.69750,2.190095e+05,1.612816e+13,2.183400e+04,1.253830e+05,0.364700
50%,566.500000,567.025000,576.900000,556.500000,567.000000,566.700000,566.94000,1.010938e+06,6.832603e+13,4.406800e+04,5.017560e+05,0.511000
75%,1242.200000,1243.312500,1263.000000,1221.650000,1242.900000,1242.400000,1242.66250,3.019851e+06,1.863835e+14,7.893550e+04,1.452233e+06,0.638400
max,32861.950000,33399.950000,33480.000000,32468.100000,32849.000000,32861.950000,32975.24000,4.810589e+08,3.564334e+16,1.643015e+06,2.325307e+08,1.000000


Dropping columns 'ISIN Code', 'Company Name', 'Industry' from dataframe as they are not required to predict the prices of stocks.

In [70]:
df.drop(['ISIN Code','Company Name','Industry'],axis=1,inplace=True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366.0,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338.0,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121.0,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762.0,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470.0,2.875200e+14,NaN,816123.0,0.2741


Displaying the different Symbols of shares in the NIFTY 50 from the dataframe

In [71]:
df['Symbol'].unique()

array(['MUNDRAPORT', 'ADANIPORTS', 'ASIANPAINT', 'UTIBANK', 'AXISBANK',
       'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAUTOFIN', 'BAJFINANCE', 'BHARTI',
       'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DRREDDY',
       'EICHERMOT', 'GAIL', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HEROHONDA', 'HEROMOTOCO', 'HINDALC0', 'HINDALCO', 'HINDLEVER',
       'HINDUNILVR', 'ICICIBANK', 'INDUSINDBK', 'INFOSYSTCH', 'INFY',
       'IOC', 'ITC', 'JSWSTL', 'JSWSTEEL', 'KOTAKMAH', 'KOTAKBANK', 'LT',
       'MARUTI', 'M&M', 'NESTLEIND', 'NTPC', 'ONGC', 'POWERGRID',
       'RELIANCE', 'SBIN', 'SHREECEM', 'INFRATEL', 'SUNPHARMA',
       'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'UPL', 'VEDL', 'WIPRO', 'ZEEL', 'TELCO', 'TISCO', 'UNIPHOS',
       'SESAGOA', 'SSLT', 'ZEETELE'], dtype=object)

Checking number of NULL values in the dataset

In [72]:
df.isna().sum()

Date                      50
Symbol                     0
Series                     0
Prev Close                50
Open                      50
High                      50
Low                       50
Last                      50
Close                     50
VWAP                      50
Volume                    50
Turnover                  50
Trades                114898
Deliverable Volume     16127
%Deliverble            16127
dtype: int64

As observed from above 'Date', 'Prev Close', 'Open', 'High', 'Low', 'Last, 'Close', 'VWAP', 'Volume', 'Turnover' columns have same amount of NULL values, that is 50 NULL values. It is possible that the 50 NULL values in the above columns belong to the same 50 rows, rather than being randomly scattered across the dataframe.

In [73]:
#displaying the rows which have NULL values in the 'Date' column
df[df['Date'].isna()]

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,NaT,ADANIPORTS,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,ASIANPAINT,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,AXISBANK,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,BAJAJ-AUTO,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,BAJAJFINSV,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaT,BAJFINANCE,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaT,BHARTIARTL,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaT,BPCL,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaT,BRITANNIA,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaT,CIPLA,EQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The above rows can be dropped as crucial information is missing or NULL.

In [74]:
#dropping rows in df which have NULL values in the 'Date' column
df.drop(df[df['Date'].isna()].index, inplace = True)
df.isna().sum()

Date                       0
Symbol                     0
Series                     0
Prev Close                 0
Open                       0
High                       0
Low                        0
Last                       0
Close                      0
VWAP                       0
Volume                     0
Turnover                   0
Trades                112398
Deliverable Volume     14372
%Deliverble            14372
dtype: int64

In [75]:
#displaying the rows which have NULL values in the 'Trades' column
df[df['Trades'].isna()]

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
50,2008-02-07,MUNDRAPORT,EQ,807.50,825.00,830.00,792.00,795.90,796.25,809.53,212932.0,1.723758e+13,NaN,106275.0,0.4991
51,2008-02-08,MUNDRAPORT,EQ,796.25,810.00,830.00,765.15,786.00,784.05,781.48,285025.0,2.227425e+13,NaN,154857.0,0.5433
52,2008-02-11,MUNDRAPORT,EQ,784.05,785.00,785.00,695.00,699.00,711.20,736.23,223955.0,1.648826e+13,NaN,118002.0,0.5269
53,2008-02-12,MUNDRAPORT,EQ,711.20,725.00,734.95,655.60,689.00,681.30,681.38,303409.0,2.067358e+13,NaN,187180.0,0.6169
54,2008-02-13,MUNDRAPORT,EQ,681.30,815.90,815.90,664.00,678.00,670.95,681.68,214900.0,1.464921e+13,NaN,108761.0,0.5061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,2011-05-25,ZEEL,EQ,135.95,136.15,139.70,133.00,137.95,137.35,137.46,2099385.0,2.885779e+13,NaN,1051299.0,0.5008
2846,2011-05-26,ZEEL,EQ,137.35,138.80,143.20,136.75,138.35,138.10,139.89,4914276.0,6.874777e+13,NaN,3031206.0,0.6168
2847,2011-05-27,ZEEL,EQ,138.10,139.00,139.75,137.00,139.00,138.75,138.73,2369073.0,3.286683e+13,NaN,1662276.0,0.7017
2848,2011-05-30,ZEEL,EQ,138.75,139.00,139.00,135.70,135.70,136.15,137.07,846435.0,1.160244e+13,NaN,611624.0,0.7226


In [79]:
#displaying the rows which have NULL values in the 'Deliverable Volume' column
#the number of NULL values in 'Deliverable Volume' and '%Deliverble' are same as %Deliverble is obtained from Deliverable Volume
df[df['Deliverable Volume'].isna()]

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
50,2000-03-14,ASIANPAINT,EQ,399.95,394.05,415.00,391.00,400.00,400.15,400.59,21141.0,8.468868e+11,NaN,NaN,NaN
51,2000-03-15,ASIANPAINT,EQ,400.15,400.00,419.95,400.00,410.00,407.45,411.31,3251.0,1.337160e+11,NaN,NaN,NaN
52,2000-03-16,ASIANPAINT,EQ,407.45,415.00,415.00,405.00,408.50,410.00,409.91,58313.0,2.390332e+12,NaN,NaN,NaN
53,2000-03-21,ASIANPAINT,EQ,410.00,407.00,425.00,407.00,425.00,418.05,412.84,6582.0,2.717330e+11,NaN,NaN,NaN
54,2000-03-22,ASIANPAINT,EQ,418.05,444.00,449.00,420.00,424.00,422.50,430.92,5524.0,2.380393e+11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,2007-02-12,ZEETELE,EQ,361.30,225.25,319.40,225.25,257.00,258.35,265.73,4429894.0,1.177153e+14,NaN,NaN,NaN
1786,2007-02-13,ZEETELE,EQ,258.35,260.00,276.95,252.25,272.55,267.95,269.91,2689950.0,7.260378e+13,NaN,NaN,NaN
1787,2007-02-14,ZEETELE,EQ,267.95,272.00,277.00,256.50,265.50,265.50,263.25,2820444.0,7.424754e+13,NaN,NaN,NaN
1788,2007-02-15,ZEETELE,EQ,265.50,269.90,273.50,265.10,268.00,268.25,268.78,3516748.0,9.452372e+13,NaN,NaN,NaN


Since %Deliverble and Deliverable Volume depict the same entity, we may drop one of the columns

In [80]:
#dropping 'Deliverable Volume' column
df.drop(['Deliverable Volume'],axis=1,inplace=True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,%Deliverble
50,2008-02-07,MUNDRAPORT,EQ,807.50,825.0,830.00,792.00,795.9,796.25,809.53,212932.0,1.723758e+13,NaN,0.4991
51,2008-02-08,MUNDRAPORT,EQ,796.25,810.0,830.00,765.15,786.0,784.05,781.48,285025.0,2.227425e+13,NaN,0.5433
52,2008-02-11,MUNDRAPORT,EQ,784.05,785.0,785.00,695.00,699.0,711.20,736.23,223955.0,1.648826e+13,NaN,0.5269
53,2008-02-12,MUNDRAPORT,EQ,711.20,725.0,734.95,655.60,689.0,681.30,681.38,303409.0,2.067358e+13,NaN,0.6169
54,2008-02-13,MUNDRAPORT,EQ,681.30,815.9,815.90,664.00,678.0,670.95,681.68,214900.0,1.464921e+13,NaN,0.5061


Moreover, from the previous display of the rows which have NULL values in the 'Deliverable Volume' column, we saw that the same rows also had NULL values in the 'Trades' column. The main difference between traded volume and delivery volume is that traded volume is the total number of shares traded (bought and sold) in a day and delivery volume is the shares that get transfers from one Demat account to the other and are not squared off within a day. 47.78% of the 'Trades' column has NULL values, hence it is better to drop the column.

In [81]:
#dropping 'Trades' column
df.drop(['Trades'],axis=1,inplace=True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,%Deliverble
50,2008-02-07,MUNDRAPORT,EQ,807.50,825.0,830.00,792.00,795.9,796.25,809.53,212932.0,1.723758e+13,0.4991
51,2008-02-08,MUNDRAPORT,EQ,796.25,810.0,830.00,765.15,786.0,784.05,781.48,285025.0,2.227425e+13,0.5433
52,2008-02-11,MUNDRAPORT,EQ,784.05,785.0,785.00,695.00,699.0,711.20,736.23,223955.0,1.648826e+13,0.5269
53,2008-02-12,MUNDRAPORT,EQ,711.20,725.0,734.95,655.60,689.0,681.30,681.38,303409.0,2.067358e+13,0.6169
54,2008-02-13,MUNDRAPORT,EQ,681.30,815.9,815.90,664.00,678.0,670.95,681.68,214900.0,1.464921e+13,0.5061


6.109% of '%Deliverble' is NULL values and hence can be imputed.

Imputing missing data in '%Deliverble' column

In [82]:
#Method 1: Using ffill() 
df1 = df.ffill()
df1.isna().sum()

Date           0
Symbol         0
Series         0
Prev Close     0
Open           0
High           0
Low            0
Last           0
Close          0
VWAP           0
Volume         0
Turnover       0
%Deliverble    0
dtype: int64

In [83]:
#Method 2: Using bfill()
df2 = df.bfill()
df2.isna().sum()

Date           0
Symbol         0
Series         0
Prev Close     0
Open           0
High           0
Low            0
Last           0
Close          0
VWAP           0
Volume         0
Turnover       0
%Deliverble    0
dtype: int64

In [84]:
#Method 3: Using Interpolate() Method
df3 = df
df3['%Deliverble'] = df['%Deliverble'].interpolate()
df3.isna().sum()

Date           0
Symbol         0
Series         0
Prev Close     0
Open           0
High           0
Low            0
Last           0
Close          0
VWAP           0
Volume         0
Turnover       0
%Deliverble    0
dtype: int64

In [91]:
#Method 4: using iterative imputer

#importing required python libraries
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

df4 = df
imp_mean = IterativeImputer(random_state=0)
#df4[%Deliverble] = imp_mean.fit_transform(df4)

Outlier analysis - fit model, find squared error, set threshold for error, remove points that exceed the threshold.

# Data Visualisation